In [1]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 0.1.4


In [2]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

Found the config file in: /home/jayaubuntudsvm/Desktop/AKSDeploymentTutorial/Keras_Tensorflow/aml_config/config.json
jayavienna
jayavienna
eastus2
edf507a2-6235-46c5-b560-fd463ba2e771


In [3]:
# choose a name for the run history container in the workspace
run_history_name = 'train-on-local'

# project folder
project_folder = './sample_projects/train-on-local'

In [4]:
import os
from azureml.core import Project

project = Project.attach(workspace_object = ws,
                         history_name = run_history_name,
                         directory = project_folder)

print(project.project_directory, project.history.name, sep = '\n')

/home/jayaubuntudsvm/Desktop/AKSDeploymentTutorial/Keras_Tensorflow/sample_projects/train-on-local
train-on-local


In [5]:
%%writefile $project_folder/train.py

#Develop the model
import tensorflow as tf
import keras
from resnet152 import ResNet152
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
import numpy as np
from PIL import Image
import wget

model = ResNet152(weights='imagenet')

#model.summary( )

wget.download('https://upload.wikimedia.org/wikipedia/commons/thumb/6/68/Lynx_lynx_poing.jpg/220px-Lynx_lynx_poing.jpg')

img_path = '220px-Lynx_lynx_poing.jpg'
#print(Image.open(img_path).size)
Image.open(img_path)

img = image.load_img(img_path, target_size=(224, 224))
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = preprocess_input(img)

preds = model.predict(img)
print('Predicted:', decode_predictions(preds, top=3))

Overwriting ./sample_projects/train-on-local/train.py


In [6]:
#User-managed runs

from azureml.core.runconfig import RunConfiguration

# Editing a run configuration property on-fly.
run_config = RunConfiguration.load(project_object = project, run_config_name = "local")

run_config.environment.python.user_managed_dependencies = True
run_config.prepare_environment = True

# You can choose a specific Python environment by pointing to a Python path 
#run_config.environment.python.interpreter_path = '/home/ninghai/miniconda3/envs/sdk2/bin/python'

In [7]:
%%time 
from azureml.core.run import Run

run = Run.submit(project_object = project,
                 run_config = run_config,
                 script_to_run = 'train.py')

# Shows output of the run on stdout.
run.wait_for_completion(show_output = True)

RunId: train-on-local_1535737983

Streaming azureml-logs/60_control_log.txt

Uploading experiment status to history service.
Running: ['python', 'azureml-setup/run_script.py', 'python', 'azureml-setup/context_manager_injector.py', '-i', 'ProjectPythonPath:context_managers.ProjectPythonPath', '-i', 'OutputCollection:context_managers.RunHistory', 'train.py']
Logging experiment running status in history service.
Adding run profile attachment azureml-logs/60_control_log.txt
Uploading experiment status to history service.
Adding run profile attachment azureml-logs/80_driver_log.txt

Streaming azureml-logs/80_driver_log.txt

Using TensorFlow backend.
2018-08-31 17:53:05.917574: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2018-08-31 17:53:06.049282: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1392] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz):

In [9]:
# make sure you have helpers.py in the current folder
import helpers
print(helpers.get_run_history_url(run))

https://mlworkspace.azureml-test.net/portal/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourceGroups/jayavienna/providers/Microsoft.MachineLearningServices/workspaces/jayavienna/history/train-on-local/runs/train-on-local_1535737983


In [10]:
# System managed runs

from azureml.core.conda_dependencies import CondaDependencies

# Editing a run configuration property on-fly.
run_config = RunConfiguration.load(project_object = project, run_config_name = "local")

# Use a new conda environment that is to be created from the conda_dependencies.yml file
run_config.environment.python.user_managed_dependencies = False

# Automatically create the conda environment before the run
run_config.prepare_environment = True

# add scikit-learn to the conda_dependencies.yml file
cd = CondaDependencies()
cd.add_conda_package('scikit-learn')
cd.save_to_file(project_dir = project_folder, conda_file_path = run_config.environment.python.conda_dependencies_file)

'aml_config/conda_dependencies.yml'

In [11]:
%%time 
print("Submitting an experiment.")
run = Run.submit(project_object = project,
                 run_config = run_config, 
                 script_to_run = "train.py")

# Shows output of the run on stdout.
run.wait_for_completion(show_output = True)

Submitting an experiment.
RunId: train-on-local_1535738630

Streaming azureml-logs/60_control_log.txt

Uploading experiment status to history service.
Adding run profile attachment azureml-logs/60_control_log.txt
Running ['conda', '--version']
Creating Conda environment...
Logging experiment preparation status in history service.
Solving environment: ...working... done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.5
  latest version: 4.5.11

Please update conda by running

    $ conda update -n base -c defaults conda



ipywidgets-7.4.0     |  150 KB | ########## | 100% 
nb_conda-2.2.0       |   33 KB | ########## | 100% 
widgetsnbextension-3 |  1.7 MB | ########## | 100% 
tornado-4.5.3        |  617 KB | ########## | 100% 
chardet-3.0.4        |  189 KB | ########## | 100% 
nb_anacondacloud-1.4 |   27 KB | ########## | 100% 
python-3.6.2         | 27.0 MB | ########## | 100% 
clyent-1.2.2         |   18 KB | ########## | 100% 
pytz-2018.5          |  232 K

  Using cached https://files.pythonhosted.org/packages/69/a7/88719d132b18300b4369fbffa741841cfd36d1e637e1990f27929945b538/kiwisolver-1.0.1-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/6a/8a/718fd7d3458f9fab8e67186b00abdd345b639976bc7fb3ae722e1b026a50/pyparsing-2.2.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f7/d2/e07d3ebb2bd7af696440ce7e754c59dd546ffe1bbe732c8ab68b9c834e61/cycler-0.10.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/61/67/df71b367680e06bb4127e3df6189826d4b9daebf83c3bd5b9341c99ef528/yarl-1.2.6-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/cc/30/508a22a28dfb50cf9079cd9d0cf9b0d7dbae5afdf9823977351cbd548897/multidict-4.3.1-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/96/0f/e6357458c87fb4ed8f3df215773f3caad40968f10e05552cbd8bd28415e4/async_timeout-3.0.0-py3-none-any.whl
  Using cached h

  Using cached https://files.pythonhosted.org/packages/d2/68/416d0341e6e2f3da3794c7226c05ae8ccb9dcfbf29b8c50a1db56b2e6c9a/azure_cli_dls-0.1.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/05/39/5125d12955f8253f86e577fc22e72a6295b7a05163c1415f80820de3448c/azure_cli_dms-0.1.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d2/81/6db9a091ef0cd1c0f9ba7625188e4f6f16c6431d491f7ee3618dfe487e43/azure_cli_servicefabric-0.1.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/0c/1a/e2d253d4c3c56395959643ebcb575afafd1653db6243126b7c76b76502e4/azure_cli_billing-0.2.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f4/05/8f9fc7981f901ff9c886fd5b76fa304ef92a7f675fd14323b42b53420f90/azure_cli_consumption-0.4.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/6f/fe/e654dbbfadbc95968e22617e246ccdbdd95002918fdb5c7d1774369698e4/azure_cli_ams-0.2.3-py2.py3-none-any.whl


  Using cached https://files.pythonhosted.org/packages/8c/10/79282747f9169f21c053c562a0baa21815a8c7879be97abd930dbcf862e8/setuptools-39.1.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/9e/1f/3da43860db614e294a034e42d4be5c8f7f0d2c75dc1c428c541116d8cdab/tensorboard-1.9.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/a8/0b/f163da98d3a01b3e0ef1cab8dd2123c34aee2bafbb1c5bffa354cc8a1730/scipy-1.1.0-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/8e/cb/726134109e7bd71d98d1fcc717ffe051767aac42ede0e7326fd1787e5d64/h5py-2.8.0-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/e2/60/c557075e586e968d7a9c314aa38c236b37cb3ee6b37e8d57152b1a5e0b47/Keras_Applications-1.0.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f8/33/275506afe1d96b221f66f95adba94d1b73f6b6087cfb6132a5655b6fe338/Keras_Preprocessing-1.0.1-py2.py3-none-any.whl
  Using cach

  Using cached https://files.pythonhosted.org/packages/a2/71/8273a7eeed0aff6a854237ab5453bc9aa67deb49df4832801c21f0ff3782/contextlib2-0.5.5-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/06/0b/ce97eb31058eddaef316973b8299b737ebab0bcb5798fd5a3225d53b4455/docker-3.5.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/89/0a/29f7e2914033e2536026b8f0d7f8deb1edda68c9a93ce4757b2b1e39568b/azure_graphrbac-0.40.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e8/78/3bb4b819ae7ab81ba0759788b28671d7ae13186544c333f973e6c91a6726/textwrap3-0.9.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d7/14/2a0004d487464d120c9fb85313a75cd3d71a7506955be458eebfe19a6b1d/s3transfer-0.1.13-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/9e/ef/5d6d9995946379e018bca8353271a3d7068daf5892f4bcdddd59faded358/botocore-1.11.5-py2.py3-none-any.whl
  Using cached https://files.pythonhos

  Using cached https://files.pythonhosted.org/packages/e6/35/f187bdf23be87092bd0f1200d43d23076cee4d0dec109f195173fd3ebc79/mock-2.0.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/31/88/ba8d8684a8a27749250c66ff7c2b408fdbc29b50da61200338ff9b2607bf/argcomplete-1.9.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/db/c8/7dcf9dbcb22429512708fe3a547f8b6101c0d02137acbd892505aee57adf/colorama-0.3.9-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/fb/34/1e80e4a06020f7dfc7eee11387e400435236f992f80eaa6ed9b3d8252211/humanfriendly-4.16.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/3e/db/cb7b6656e0e7387637ce850689084dc0b94b44df31cc52e5fc5c2c4fd2c1/paramiko-2.4.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/bc/49/1c69f851ea9d17db920d07dee62f2be3bdca52ab0a65728324c797c99285/knack-0.4.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/pack

  Using cached https://files.pythonhosted.org/packages/6a/0b/fe3a2d47066005c928056f60ed308036485a7d8bcfa6595d4f7a45f1b938/websocket_client-0.51.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e0/82/05c88b9e8f34deb510fd8ea63ecc04c820ffc3eb3ef578ff8243a7460ebe/azure_datalake_store-0.0.27-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/a6/68/aba86e698f24f25f70d9ee2b15a2ca0d71b822932ace8ba96e5beb12efdd/azure_mgmt_datamigration-0.1.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/6e/06/fafe8b5d881cfa68927e61557c8419dcfacb93e07f4ab17cc60959707a53/azure_mgmt_servicefabric-0.2.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ff/32/cf501356a6f7ac0299141e7f7736cdf5cba60a5a72e030ab036e43b9aed4/azure_mgmt_compute-4.0.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/0f/24/5106287ea0f6562d965afb055e3c6c0da058f7844a70464e67bab56c6c4b/azure_mgmt_billing-0.2.0

  Using cached https://files.pythonhosted.org/packages/20/c4/12e3e56473e52375aa29c4764e70d1b8f3efa6682bef8d0aae04fe335243/Werkzeug-0.14.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/c9/e3/48b98f929290b0cd0aaa4707ecabb8aaeb267e8ef628af907f1d1c506ec7/azure_mgmt_nspkg-2.0.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/88/ec/f598b633c3d5ffe267aaada57d961c94fdfa183c5c3ebda2b6d151943db6/backports.weakref-1.0.post1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/94/e7/c250d122992e1561690d9c0f7856dadb79d61fd4bdd0e598087dce607f6c/requests_oauthlib-1.0.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/9b/9f/b36f7774ff5ea8e428fdcfc4bb332c39ee5b9362ddd3d40d9516a55221b2/isodate-0.6.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ea/bf/7e70aeebc40407fbdb96fa9f79fc8e4722ea889a99378303e3bcc73f4ab5/docker_pycreds-0.3.0-py2.py3-none-any.whl
  Using cached h

  Found existing installation: setuptools 40.2.0
    Uninstalling setuptools-40.2.0:
      Successfully uninstalled setuptools-40.2.0
#
# To activate this environment, use
#
#     $ conda activate /home/jayaubuntudsvm/.azureml/envs/azureml_0961f65f05e3aaa4101dad5b3861013d
#
# To deactivate an active environment, use
#
#     $ conda deactivate





Running: ['/home/jayaubuntudsvm/.azureml/envs/azureml_0961f65f05e3aaa4101dad5b3861013d/bin/python', 'azureml-setup/run_script.py', '/home/jayaubuntudsvm/.azureml/envs/azureml_0961f65f05e3aaa4101dad5b3861013d/bin/python', 'azureml-setup/context_manager_injector.py', '-i', 'ProjectPythonPath:context_managers.ProjectPythonPath', '-i', 'OutputCollection:context_managers.RunHistory', 'train.py']
Logging experiment running status in history service.
Uploading experiment status to history service.
Adding run profile attachment azureml-logs/80_driver_log.txt

Streaming azureml-logs/80_driver_log.txt

Using TensorFlow backend.
2018-08-31 18:05:53.4109

In [12]:
# import helpers.py
import helpers
print(helpers.get_run_history_url(run))

https://mlworkspace.azureml-test.net/portal/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourceGroups/jayavienna/providers/Microsoft.MachineLearningServices/workspaces/jayavienna/history/train-on-local/runs/train-on-local_1535738630
